# Java 5 — RAG (Gjenerim i përforcuar nga kërkimi)

**Objektivat:**
- Të kombinohet retrieval + generation.
- Të prodhohen përgjigje të bazuara në tekst me citime.
- Të zbatohet refuzimi kur retrieval është i dobët.

**Detyra (Assignment):**
- Implemento funksionin `answer_with_rag(question)`.
- Kthe përgjigje + citime (doc_id + chunk).
- Testo me 10 pyetje dhe 2 pyetje që duhen refuzuar.

---

# Doctrine & SOP Assistant (Fiktiv) — Google Colab Notebook

**Kujdes / Safety (e detyrueshme):**
- Ky projekt është *arsimor* dhe përdor dokumente **fiktive, të paklasifikuara**.
- Asistenti **nuk** jep këshilla taktike/operacionale.
- Nëse informacioni nuk gjendet në dokumente → **refuzim i sigurt** (“Nuk mbulohet nga dokumentet e disponueshme.”).

---

## 📁 Struktura e të dhënave (rekomandim)

Vendosni PDF-të në:
- `data/raw_pdfs/`

Emrat e rekomanduar (shembull):
- `AAF-TRN-001_SOP_e_Stërvitjes.pdf`
- `AAF-SAFE-010_Rregullat_e_Sigurisë_në_Poligon.pdf`
- `AAF-LOG-004_Siguria_dhe_Kontrolli_i_Mjeteve.pdf`
- `AAF-CMD-007_Zinxhiri_Komandues_dhe_Raportimi_i_Incidenteve.pdf`
- `AAF-GRD-002_Procedurat_e_Sherbimit_te_Rojes.pdf`
- `NATO_Permbledhje_Doktrinore_Edukative.pdf`
- `AAF-IDX-000_Indeksi_Doktrinor_dhe_Procedural.pdf`
- `AAF-ADM-012_Rregulla_Administrative_dhe_Arkivimi.pdf`
- `AAF-RPT-009_Udhezime_te_Pergjithshme_per_Raportim.pdf`


## 0) Setup & Import

In [ ]:
!pip -q install sentence-transformers chromadb pypdf faiss-cpu rapidfuzz

import os, re, json, math
from pathlib import Path


## 1) Ngarko PDF-të (zgjidh një mënyrë)

### Opsioni A — Upload direkt në Colab

In [ ]:
from google.colab import files
uploaded = files.upload()  # zgjidh PDF-të nga kompjuteri
os.makedirs("data/raw_pdfs", exist_ok=True)
for fn in uploaded.keys():
    Path(fn).replace(Path("data/raw_pdfs")/fn)
print("U ngarkuan:", list(uploaded.keys()))


### Opsioni B — Mount Google Drive (rekomanduar për projekte)

In [ ]:
# nga google.colab import drive
# drive.mount('/content/drive')
# Pastaj kopjoni/ndryshoni rrugën sipas vendit ku i keni ruajtur PDF-të, p.sh.:
# !mkdir -p data/raw_pdfs
# !cp -n /content/drive/MyDrive/doctrine_sop_dataset/raw_pdfs/*.pdf data/raw_pdfs/


## 2) Retrieval (Chroma) — përdor koleksionin nga Java 4

In [ ]:
import chromadb
from chromadb.utils import embedding_functions

client = chromadb.PersistentClient(path="data/chroma")
ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="sentence-transformers/all-MiniLM-L6-v2")
col = client.get_or_create_collection(name="doctrine_sop", embedding_function=ef)

def retrieve(query, k=4):
    res = col.query(query_texts=[query], n_results=k)
    ctx = []
    for i in range(len(res["ids"][0])):
        ctx.append({
            "id": res["ids"][0][i],
            "doc_id": res["metadatas"][0][i].get("doc_id") if res.get("metadatas") else None,
            "text": res["documents"][0][i]
        })
    return ctx


## 3) Generator (zgjidh një model)

Në Colab, për thjeshtësi mund të përdorni:
- API (nëse lejohet në kurs), ose
- një model të vogël lokal (p.sh. `microsoft/phi-3-mini-4k-instruct`) me `transformers`.

Më poshtë jepet një **stub**. Plotësojeni sipas zgjedhjes së kursit.

In [ ]:
# STUB: vendosni generatorin tuaj këtu.
# Shembull i thjeshtë: kthe vetëm tekstin e përmbledhur nga konteksti (pa LLM) për demonstrim.

def generate_answer(question: str, contexts: list) -> str:
    # TODO: zëvendëso me LLM
    joined = "\n\n".join([f"[{c['id']}] {c['text']}" for c in contexts])
    return "PËRGJIGJE (stub) bazuar në kontekst:\n\n" + joined[:1500]


## 4) RAG Answer + Citime + Refuzim

In [ ]:
def answer_with_rag(question: str, k=4, min_contexts=1):
    ctx = retrieve(question, k=k)
    if len(ctx) < min_contexts:
        return {
            "answer": "Nuk mund të përgjigjem bazuar në dokumentet e disponueshme. Ju lutem konsultoni dokumentin origjinal ose eprorin përkatës.",
            "citations": []
        }
    ans = generate_answer(question, ctx)
    cits = [{"id": c["id"], "doc_id": c.get("doc_id")} for c in ctx]
    return {"answer": ans, "citations": cits}

test_q = "Cila komandë jepet në rast emergjence në poligon?"
out = answer_with_rag(test_q)
print(out["answer"][:600])
print("CITATIONS:", out["citations"])


## 5) Dorëzim

- 10 pyetje me përgjigje + citime.
- 2 pyetje që kërkojnë informacion që mungon → refuzim i saktë.